# Parse from JSON file

In [ ]:
import json
with open ("F:/genome_test/ncbi_dataset/data/assembly_data_report.jsonl", 'r') as json_file:
    json_file_as_list = json_file.readlines()

output = "Accession\tNumber of Scaffolds\tContig N50\n"
for accession_detail_line in json_file_as_list:
    accession_detail = json.loads(accession_detail_line)
    contig_n50 = (int((accession_detail["assemblyStats"]["contigN50"]))/1000000)
    accession_number = ((accession_detail["accession"]))
    scaffold_number = int(accession_detail["assemblyStats"]["numberOfScaffolds"])
#     print(accession_number,scaffold_number,contig_n50 )
    output = output + (f"{accession_number}\t{scaffold_number}\t{contig_n50}\n")
    
with open ("F:/genome_test/temp_details.txt",'w') as out_file:
    out_file.write(output)

# Parse from Txt file

In [ ]:
import os


with open("J:/genome_test/test2/gcf.txt",'r') as list_file:
    
    list_of_accessions = list_file.readlines()

output = "Accession\tScaffold_number\tContig_N50\n"
for i in range(len(list_of_accessions)):
    accession = list_of_accessions[i].rstrip()
#     print(accession)
#     break
    
    location = f"J:/genome_test/test2/genbank/invertebrate/{accession}"
    try:
        files_in_accession_folder = os.listdir(location)
    except FileNotFoundError:
        output += f"{accession}\tN/A\tN/A\n"
        continue
    for file in files_in_accession_folder:
        if "assembly_stats" in file:
           with open(f"{location}/{file}", 'r') as assembly_stat_file:
                stat_file_list = assembly_stat_file.readlines()
                break
    
    for lines in stat_file_list:
        if "all	all	all	all	scaffold-count" in lines:
            scaffold_count = lines.split("	")[-1].rstrip()
        if "all	all	all	all	contig-N50" in lines:
            contig_n50 = int(lines.split("	")[-1].rstrip())/1000000
#     print(scaffold_count, contig_n50)
    output += f"{accession}\t{scaffold_count}\t{contig_n50}\n"
    
with open("J:/genome_test/test2/accession_details.txt",'w') as output_file:
    output_file.write(output)
    


# Code for extraction and makeblastdb


In [ ]:
import os

location = "J:/Genomes_2023-12-26"

list_of_species = os.listdir(location)
output = '#!/bin/zsh\n'
for species in list_of_species:
    files_list = os.listdir(f"{location}/{species}")
    for files in files_list:
        if ".gz" in files:
            output += f"echo {species} \ngzip -d /mnt/j/Genomes_2023-12-26/{species}/{files}\n"
            
#             print(species)

print(output)
            
# with open("J:/My Drive/Circadian Rhythm Genes Project/1.Codes/bash scripts/unzip.sh", "w") as out_file:
#     out_file.write(output)
    


In [ ]:
import os

location = "J:/Genomes_2023-12-26"

list_of_species = os.listdir(location)
output = '#!/bin/zsh\n'
for species in list_of_species:
    files_list = os.listdir(f"{location}/{species}")
    for files in files_list:
        if "genomic.fna" in files[-11:]:
            output += f"echo {species} \nmakeblastdb -in /mnt/j/Genomes_2023-12-26/{species}/{files} -dbtype nucl\n"
    
#             print(species)

# print(output)
            
with open("K:/My Drive/Circadian Rhythm Genes Project/1.Codes/bash scripts/makeblastdb.sh", "w") as out_file:
    out_file.write(output)
    

# Parse gene table from NCBI

In [17]:
import re 
from Bio import Entrez, SeqIO, Align
from Bio.Align import substitution_matrices
from io import StringIO

aligner = Align.PairwiseAligner()
aligner.mode = 'global'
aligner.substitution_matrix = substitution_matrices.load("BLOSUM62")
aligner.open_gap_score = -10
aligner.extend_gap_score = -0.5
aligner.target_end_gap_score = 0.0
aligner.query_end_gap_score = 0.0


Entrez.email = 'saubaral@gmail.com'


def get_protein_sequence(ncbi_protein_id):
    handle = Entrez.efetch(db="protein", id=ncbi_protein_id, rettype="fasta", retmode="text")
    record = handle.read()
    fasta_io = StringIO(record) 
    protein_sequence_element = SeqIO.parse(fasta_io, "fasta")
    for record in protein_sequence_element:
        return(record)


species = "1.Per period[Bombyx mori]"
bombyx_protein_accession = "NP_001036975.1"
bombyx_protein = get_protein_sequence(bombyx_protein_accession)


with open(f"H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/0.Species/{species}/Gene_table.txt","r") as gene_table_file:
    gene_table_list = gene_table_file.readlines()

    
exon_list = "Exon1,Exon2,Exon3,Exon4,Exon5,Exon6,Exon7,Exon8,Exon9,Exon10,Exon11,Exon12,Exon13,Exon14,Exon15,Exon16,Exon17,Exon18,Exon19,Exon20,Exon21,Exon22,Exon23,Exon24,Exon25,Exon26,Exon27,Exon28,Exon29,Exon30,Exon31,Exon32,Exon33,Exon34".split(",")
i = 0
switch = 0
output = "mRNA Accession\tExon Number\tCoding Length\tGenome Start Point\tGenome Stop Point\n" 
output2 = "mRNA Accession\tProtein Accession\tExon Number\tAlignment Score with Bombyx mori\n"
exon_counter = 0

while i < (len(gene_table_list)):
    
    if "Exon table for  mRNA  " in gene_table_list[i]:
        gene_table_line = gene_table_list[i]
        while re.search("[ ]{2,}", gene_table_line):
            gene_table_line = gene_table_line.replace("  "," ")
            
        mrna_accession = gene_table_line.split(" ")[4]
        protein_accession = gene_table_line.split(" ")[7].rstrip()
#         print(get_protein_sequence(protein_accession))
#         alignment_score = aligner.score(bombyx_protein.seq, get_protein_sequence(protein_accession).seq)
#         print(alignment_score)
        
#         for alignment in alignments:
#             print("Score = %.1f:" % alignment.score)
#         assert False
        
#         print(f"{mrna_accession}\t{protein_accession}\t{alignment_score}", end = "\t")
        
        switch = 1
        i += 3
    if switch == 1:

        gene_table_line = gene_table_list[i]
        while re.search("[\t]{2,}", gene_table_line):
            gene_table_line = gene_table_line.replace("\t\t","\t")
#         print(len(gene_table_line.split("\t")))
        if len(gene_table_line.split("\t")) == 7 or len(gene_table_line.split("\t")) == 6:
            
            coding_exon_length =  ((gene_table_line.split("\t")[5]))
            genome_start_point,genome_end_point = ((gene_table_line.split("\t")[1]).split("-"))
            print((gene_table_line.split("\t")[1]).split("-"))
            output += f"{mrna_accession}\t{exon_list[exon_counter]}\t{coding_exon_length.rstrip()}\t{genome_start_point}\t{genome_end_point}\n"
            
        
            exon_counter += 1
        
        if switch == 1 and re.search("^mRNA",gene_table_list[i]):
            switch = 0
            print(exon_counter)
#             output2 += f"{mrna_accession}\t{protein_accession}\t{exon_counter}\t{alignment_score}\n"
            output += "\n\n"
            exon_counter = 0
            
    i += 1

with open(f"H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/0.Species/{species}/Gene_table_processed1.tsv",'w') as out_file:
    out_file.write(output)

with open(f"H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/0.Species/{species}/Gene_table_alignment_score1.tsv",'w') as out_file:
    out_file.write(output2)
    

['12522661', '12522573']
['12521246', '12521163']
['12520999', '12520845']
['12514423', '12514329']
['12513351', '12513274']
['12512245', '12512114']
['12511968', '12511893']
['12510243', '12510040']
['12508701', '12508473']
['12507855', '12507744']
['12506233', '12506114']
['12505865', '12505716']
['12503906', '12503788']
['12503481', '12503392']
['12502301', '12502149']
['12501492', '12501303']
['12498481', '12498334']
['12497509', '12497396']
['12497300', '12497286']
['12496602', '12496486']
['12494243', '12494059']
['12490887', '12490759']
['12483156', '12482956']
['12482268', '12482134']
['12481645', '12481544']
['12478465', '12478217']
26
['12522661', '12522573']
['12521246', '12521163']
['12520999', '12520845']
['12514423', '12514329']
['12513351', '12513274']
['12512245', '12512114']
['12511968', '12511908']
['12510243', '12510040']
['12508701', '12508473']
['12507855', '12507744']
['12506233', '12506114']
['12505865', '12505716']
['12503906', '12503788']
['12503481', '12503392

# Download ncbi sequence using accession

In [47]:
from Bio import Entrez, SeqIO
from io import StringIO

Entrez.email = 'saubaral@gmail.com'
handle = Entrez.efetch(db="nucleotide", id="XM_022976811.1", rettype="fasta", retmode="text")
# print (handle)
record = handle.read()
print(record)
fasta_io = StringIO(record) 
record1 = SeqIO.parse(fasta_io, "fasta")
for sequences in record1:
    print(sequences.id)
# record.write(record.rstrip('\n'))

>XM_022976811.1 PREDICTED: Spodoptera litura period circadian protein (LOC111360701), transcript variant X3, mRNA
GCTCCGGCTAGCTGAACGTTTTGTGAGAAGTAACTCGTAACTCATTCGTTTTGTCGTTTGTTCTTTTATT
AGGTACTGTTAAACTTATATATTCGTCGATCAATATTAGATGTCGAAAAGTTTAAAAGTGCTATAATTTG
TTACGATAATAACGTACTAATTACTGTTTGAAACTTGATTAACTAAAGTGACCCTTGTGAAAACGGAATA
TTCACCTACTCTAAACAAGGATAACGTGATTTCGGCATTGTTTCTTTTTAAAGACCATCAATGCAACATG
GACAACTTGGATGATTCAGAGAACAATGCGAAGATTTCAGATTCGGCATACTCCAACAGTTGCAGTAATA
GTCAATCTAGAAGGAGTCATAGTTCTAAGTCAACACATTCTGGAAGCAATTCCAGTGGCAGCAGTGGCTA
TGGCGGGAAACCCTCCACATCTGGAAGCAGCAACAACTTGGGTCAACCACCTAAAAAAGATAAGGAACCC
AAGAAAAAGAAGCCTCAAATTGAAACAATGGTCCCAGATGTAGTTGTGGAAACGGAGGCAAGGGCGCCTG
AGCCGGCCCCTACATTTGATGCTCCGAAAGAAGATCAGCTAGATGTTATCCCGTCGCTACCACCTGCGCA
TGCGCAATCAGAGAAGGGAGTGGAGAACATGGACATATGTACCCCTGAGTTGAATACTTTAAAAGATGAG
GTGGTTTCCTGCAACATATCTCAGGTTAATCCTGCTTCAAGCTTTGTTACCGGTCGTCCCCTGCAAGCTT
CTTATTGCTCGGTACAAAGTAAACGATACCAAGAGATTGGCCGCTTCGAGATGCGATGTGAGAGCTCTGA
ACAGTTGCGTTGCAAGGATGGATTTTCATGCGTG

# Parse BLAST result

In [90]:
import csv

file_list = {}
enumerator = 0
with open("H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/1.Blast_output/T0260275016-Alignment-Descriptions.csv", 'r') as blast_out_file:
    blast_out_file_content = csv.reader(blast_out_file, delimiter=',', quotechar='"')

    
    for data in blast_out_file_content:
        file_list[str(enumerator)] = data
        enumerator += 1
#         break

output = ("\t".join(file_list[str(0)]))


for i in range(1,len(file_list)-1):
    try:
        for k in file_list[str(i)]:
#             print (k)
            if "NM_" in k:
                print(file_list[str(i)][0])
                output+= "\n"+(("\t".join(file_list[str(i)])))
    #         break
    except:
        print(f"Error{file_list[str(i)]}")

# output = "\n".join(out_list)

# with open("H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/1.Blast_output/filtered_blast.tsv", 'w') as blast_output_file:
#     blast_output_file.write(output)

Bombyx mori period (Per), mRNA


# Genomes with annotation

In [17]:
import os

list_of_genomes = os.listdir("J:/Genomes_2023-12-26")
output = "Species\tAnnotation\n"
for species in list_of_genomes:
    list_of_files= os.listdir(f"J:/Genomes_2023-12-26/{species}")
    annotation = "NO"
    for file in list_of_files:
        if "genomic.gff" in file:
            annotation = "Yes"
#             print(species)
    
    output += f"{species}\t{annotation}\n"

print(output)
    

Species	Annotation
Abrostola_tripartita	NO
Abrostola_triplasia	NO
Acentria_ephemerella	NO
Achlya_flavicornis	NO
Acleris_cristana	NO
Acleris_emargana	NO
Acleris_holmiana	NO
Acleris_literana	NO
Acleris_sparsana	NO
Acrobasis_consociella	NO
Acrobasis_repandana	NO
Acrobasis_suavella	NO
Acronicta_aceris	NO
Acronicta_leporina	NO
Acronicta_psi	NO
Adoxophyes_honmai	NO
Agonopterix_arenella	NO
Agonopterix_heracliana	NO
Agonopterix_subpropinquella	NO
Agriopis_aurantiaria	NO
Agriopis_leucophaearia	NO
Agriopis_marginaria	NO
Agriphila_geniculea	NO
Agriphila_straminella	NO
Agriphila_tristella	NO
Agrochola_circellaris	NO
Agrochola_litura	NO
Agrochola_lota	NO
Agrochola_lychnidis	NO
Agrochola_macilenta	NO
Agrotis_clavis	NO
Agrotis_exclamationis	NO
Agrotis_ipsilon	NO
Agrotis_puta	NO
Alcis_repandata	NO
Allophyes_oxyacanthae	NO
Alsophila_aescularia	NO
Amorpha_juglandis	NO
Amphipoea_lucens	NO
Amphipoea_oculea	NO
Amphipyra_berbera	NO
Amphipyra_tragopoginis	NO
Amyelois_transitella	Yes
Anania_crocealis	NO
Anani

# Parse fasta query file

In [24]:
from Bio import SeqIO
from Bio.Seq import Seq

def print_codon_translated(exon_sequence,exon_counter,exon_overhang):
    counter = 0
    codon_counter = 0
    protein_print_overhang = 0
    if exon_overhang != 0:
        print(f"{exon_sequence[counter : counter + exon_overhang]} ", end = '')
        protein_print_overhang = 1
        
    counter = counter + exon_overhang
    start_position = counter
    while counter < len(exon_sequence):
        print(f"{exon_sequence[counter : counter + 3]} ", end = '')
        counter += 3
        codon_counter += 1
        if codon_counter%5 == 0:
            print("\n")
            if protein_print_overhang == 1:
                print (f"{' '*exon_overhang} ", end = '')
                protein_print_overhang = 0
            
            while start_position < counter:
                print(f" {Seq(exon_sequence[start_position : start_position + 3]).translate()}  ", end = '')
                start_position += 3
            print("\n")
    
    print("\n")
    if protein_print_overhang == 1:
        print (f"{' '*exon_overhang} ", end = '')
        protein_print_overhang = 0

    while start_position < counter:
        print(f" {Seq(exon_sequence[start_position : start_position + 3]).translate()}  ", end = '')
        start_position += 3
    print("\n")
    return


query_rna_file = SeqIO.parse("H:/My Drive/Circadian Rhythm Genes Project/4.Queries for Blast/1.Period/0.Species/1.Bombyx mori/Query_rna.txt", 'fasta')
exon_counter = 1
full_sequence = ''
exon_overhang = 0
exon_3_overhang = 0
exon_5_overhang = 0

output_protein_query = ''

for sequences in query_rna_file:
    
    exon_sequence = str(sequences.seq)
    print(f"Exon {exon_counter}, Length = {len(sequences.seq)}\nName:{sequences.id}\n\n")
    
    print_codon_translated(exon_sequence,exon_counter,exon_overhang)
    
    exon_without_overhang_length = len(exon_sequence) - exon_overhang
    exon_5_overhang = exon_overhang
    exon_overhang = 3 - exon_without_overhang_length % 3
    if exon_overhang == 3:
        exon_overhang = 0
   
    print(exon_overhang)
    translated = (sequences.seq[exon_5_overhang:]).translate()
    print (f"\ntranslated = {translated}\ntranslated_length = {len(translated)}\n")
    
    output_protein_query += f">Exon_{exon_counter}_Bombyx_mori_NM_001043510.1\n{translated}\n\n"
    exon_counter += 1
    
# print(output_protein_query)

with open("H:/My Drive/Circadian Rhythm Genes Project/4.Queries for Blast/1.Period/0.Species/1.Bombyx mori/Query_protein.txt", 'w') as out_file:
    out_file.write(output_protein_query)
    
    
    
#     break
#     counter = 0
# #     while counter < len(exon_sequence):
        
    
    
#     if len(sequences.seq)%3 != 0:
#         print(sequences.seq, len(sequences.seq))
#         break
# #         my_seq = Seq(full_sequence)

# #     print(my_seq.translate())

    
    
    
    
    


Exon 1, Length = 89
Name:NC_051358.1:c12522661-12522573


ATG GAT AAC CTT GAA 

 M   D   N   L   E  

AAC TCT GAG AAC ACT 

 N   S   E   N   T  

GAC AAA ATA TCA GAT 

 D   K   I   S   D  

TCG GCA TAC TCA AAC 

 S   A   Y   S   N  

AGC TGC AGT TAC AGT 

 S   C   S   Y   S  

CAA TCA AGA AGA AG 

 Q   S   R   R     





1

translated = MDNLENSENTDKISDSAYSNSCSYSQSRR
translated_length = 29

Exon 2, Length = 84
Name:NC_051358.1:c12521246-12521163


T CAC AGC TCA AAA TCT 

   H   S   S   K   S  

ACT CAT TCC GGG AGT 

 T   H   S   G   S  

TAT TCT AGC GGC AGT 

 Y   S   S   G   S  

AGC GGA TAC GGA GGT 

 S   G   Y   G   G  

CGA CAT GTT ACT GGC 

 R   H   V   T   G  

ACT AAC AG 

 T   N     

1

translated = HSSKSTHSGSYSSGSSGYGGRHVTGTN
translated_length = 27

Exon 3, Length = 155
Name:NC_051358.1:c12520999-12520845


T AAT AAG TTG GCT CAG 

   N   K   L   A   Q  

TCA CCA AAT AAA GAT 

 S   P   N   K   D  

AAA GAG TTA AAG AAA 

 K   E   L   K   K  

AAG AAA CAA TCA CAA 

 K   K   Q   

CCT TCC ACA TCG GAG 

 P   S   T   S   E  

GCC GCG AAC CAT CAA 

 A   A   N   H   Q  

AAT CTT CAT AAT AAT 

 N   L   H   N   N  

CCA CAG CAG TAT GCT 

 P   Q   Q   Y   A  

CCA CCT CCA A 

 P   P   P     

2

translated = MLPLYYTPMATMAPVPSTSEAANHQNLHNNPQQYAPPP
translated_length = 38

Exon 20, Length = 185
Name:NC_051358.1:c12494243-12494059


TG CAA TGT GTA TTG TAC 

    Q   C   V   L   Y  

GGT CAG CCT ATA TAC 

 G   Q   P   I   Y  

GGT CAG CCG ATG TAC 

 G   Q   P   M   Y  

AGT TCG CCG TTC GTG 

 S   S   P   F   V  

TAC TCA CCA ATG AAC 

 Y   S   P   M   N  

CCG CAC ACT AAC TAT 

 P   H   T   N   Y  

CCT ATG CAA CAG ACC 

 P   M   Q   Q   T  

ACG CCG CAA CCG AAT 

 T   P   Q   P   N  

GCT CAG TTC ACA CCC 

 A   Q   F   T   P  

ACA AAC ACA ATG AAT 

 T   N   T   M   N  

CCC TTG TGT TTG GCC 

 P   L   C   L   A  

AAT AGC AAC TAT GAA 

 N   S   N   Y   E  

GAG 

 E  

0

translated = QCVLYGQPIYGQPMYSSPFVYSPMNPHTNYPMQQTTPQPNAQFTPTNTMNPLCLANSNYEE
translated_length = 61

Exon

# tblastn for all annotated

In [39]:
import os

species_folder = "H:/My Drive/Circadian Rhythm Genes Project/4.Queries for Blast/1.Period/1.Blast_output"
genome_folder = "J:/Genomes_2023-12-26/"
genomes_list = os.listdir(genome_folder)
blast_command = ''

list_of_species = os.listdir(species_folder)
for species_name in list_of_species:
    if ".ini" not in species_name:
        species = species_name.split(".")[1]
#         if species in genomes_list:
#             print(species)
    if species in genomes_list:
        list_of_files_in_genome_folder = os.listdir(genome_folder + species)
        for file in list_of_files_in_genome_folder:
            if "genomic.fna" == file[-11:]:
                genome_file = file
#                 print(genome_file)
        blast_command += f'tblastn -seg no -query "/mnt/h/My Drive/Circadian Rhythm Genes Project/4.Queries for Blast/1.Period/0.Species/1.Bombyx mori/Query_protein.txt" -db  "/mnt/j/Genomes_2023-12-26/{species}/{genome_file}" -out "/mnt/h/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/1.Blast_output/{species_name}/{species}_blast_out.htm" -html\n'
    else:
        print(f"{species} not present in genome folder")

    
with open("H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/tblastn_out.txt", 'w') as out_file:
    out_file.write(blast_command)
#     print(blast_command)



Achroia_grisella not present in genome folder
Achroia_grisella_2 not present in genome folder


# Extract a specific exon

In [61]:
%reset -f
from Bio import SeqIO
from Bio.Seq import Seq
import os

def get_genome_file(genome_folder,species):
    list_of_files_in_genome_folder = os.listdir(genome_folder + species)
    for file in list_of_files_in_genome_folder:
        if "genomic.fna" == file[-11:]:
            genome_file = file
    return (genome_file)


scaffold = "NW_013528315.1"
species = "Papilio_polytes"
exon_start = 485269
exon_end = 485346
offset_5, offset_3 = 1,2
genome_folder = "J:/Genomes_2023-12-26/"
reverse = 0


# print(exon_start, exon_end)

genome_file_name = get_genome_file(genome_folder,species)

parsed_genome_file = SeqIO.parse(f"{genome_folder+species}/{genome_file_name}","fasta" )

for sequences in parsed_genome_file:
    if sequences.id == scaffold:
        exon_sequence = (str(sequences.seq)[exon_start:exon_end])

if reverse == 1:
    exon_sequence = Seq(exon_sequence).reverse_complement()
exon_sequence_translated = Seq(exon_sequence)[offset_5:].translate()

print(exon_sequence)
print(f">Exon2_{species}_{scaffold}_{exon_start}_{exon_end}\n{exon_sequence_translated}")
    








tcGAAGTCAACCCACTCTGGAAGCAATTCGAGTGGCAGTAGCGGTTATGGCGGGAAACCATCTACATCCGGTTACAG
>Exon2_Papilio_polytes_NW_013528315.1_485269_485346
RSQPTLEAIRVAVAVMAGNHLHPVT


In [126]:
def sequence_extractor(Species, Scaff, reverse_c, start, end, frame = 1, trans = 1 ):
    from Bio import SeqIO
    from Bio.Seq import Seq
    import os
    
    
    entries = os.listdir("F:/Genomes_2023-12-26/"+Species)
    for file_names  in entries:
        if ".nhr" in file_names:
            Genome_name = file_names[:-4]
            break
    #print(genome_name)
    #Genome_name = "GCA_014332785.1_AM_v1.0_genomic.fna"
    
#    reverse_c = 0 #1 = yes, 0 = no
    # for pasting 3713291	3713431
#    exon = "idh"
#    start = 3713291
         
#    end = 3713434                  
    out_Seq = ''
    fasta_file = open(("F:/Genomes_2023-12-26/"+Species+"/"+Genome_name),'r')
    for record in SeqIO.parse(fasta_file,"fasta"):
    #    print (record.id)
        if record.id == Scaff:
            sequence = str(record.seq)
            out_Seq = Seq(sequence[start-1:end])
            if reverse_c == 1:
                out_Seq = str(out_Seq)
                dna_seq = Seq(out_Seq)
                out_Seq = dna_seq.reverse_complement()
    if len(out_Seq) < 10000: #fixing error due to mistake in typing the coordinate, change this for longer sequence
#        print (out_Seq)
        if frame == 1 :
            out_trans = out_Seq[0:]
        if frame == 2 :
            out_trans = out_Seq[1:]
        if frame == 3 :
            out_trans = out_Seq[2:]
#        return (out_trans.translate()) 
    else:
        print ("too long")
        assert(False)
        
        
    #    break
    fasta_file.close()
    if trans == 1:
        return (out_trans.translate())
    else:
        return(out_Seq)
#print(sequence_extractor("Carposina_sasakii", "CP053153.1", 1, 10055832, 10056257 ))

In [124]:
# rom sequence_extraction_called import *


# phylo_species = open("C:/Users/sauba/Desktop/Work_Stuff/MRJP/0.for_automation_temp_files/phylo_species.txt",'r')
# species_list = phylo_species.readlines()
species_list = ["2.Galleria_mellonella"]
#print(species_list)
query_name_list = ["Exon_1","Exon_2","Exon_3","Exon_4","Exon_5","Exon_6","Exon_7","Exon_8","Exon_9","Exon_10","Exon_11","Exon_12","Exon_13","Exon_14","Exon_15","Exon_16","Exon_17","Exon_18","Exon_19","Exon_20","Exon_21","Exon_22","Exon_23","Exon_24"]
#species_to_test = ["Amyelois_transitella","Bicyclus_anynana","Helicoverpa_armigera","Manduca_sexta","Zerene_cesonia",]
#query_length_list = [144,40,75,38,27,37,47]
seq_modi = [[0,2],[1,2],[1,1],[2,0],[0,0],[0,0],[0,1],[2,1],[2,2],[1,0],[0,0],[0,0],[0,2],[1,2],[1,2],[1,0],[0,1],[2,1],[2,1],[2,0],[0,0],[0,0],[0,0],[0,0]]
#query_length_list = [90,66,49,61,64,44,35]
#seq_modi = [[0,0,90],[0,0,66],[0,1,49],[2,2,61],[1,2,64],[1,0,44],[0,0,35],]
#try:
for species_name in species_list:
    try:
        print(species_name)
        header = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length\n" 
        Output_Sequence = header
        scaff = "Intial_value"
        scaff_old = "Intial_value"
        old_end = 0
        for i in range(len(query_name_list)):
            query_name = query_name_list[i]
            Length_switch = "0"
    #        query_length = query_length_list[i]
            tblast_out = open("H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/1.Blast_output/"+species_name+"/Galleria_mellonella_blast_out.txt",'r')
            lines_in_file = tblast_out.readlines()
            #print(lines_in_file)
            result_section_switch = 0
            start_coor_switch = 0
            query_start_coor_switch = 0
            stop_coor_switch = 0
            error = "N"
            break_switch = 0
    #        scaff = ''
            start = 0
            stop = 0
            start_coor = 0
            stop_coor = 0
            
            for lines in lines_in_file:
                
                if query_name in lines:
                    result_section_switch = 1
                    query_species_split = lines.split(" ")[1].split("_")
                    query_species = str(query_species_split[1]+"_"+query_species_split[2].rstrip())
#                     print(query_species)
#                     print(query_name)
    #                break
                if result_section_switch == 1 and "Lambda" in lines:
                    result_section_switch == 0
                    break
                    
    #                if result_section_switch == 1 and query_species not in species_to_test:
    #                    print(f"{species_name.rstrip()} has query {query_species}")
    #                    break_switch = 1
    #                    break
                    
                if result_section_switch == 1:
                    if "Length=" in lines and Length_switch == "0":
                        query_length = int(lines.split("=")[1][:-1])
                        Length_switch = 1
    #                    print(query_length)
    #                    break
                    if ("Score" in lines or ">" in lines) and (start_coor_switch == 1):
        #                print (lines)
                        break
                    if ">" in lines:
                        
                        scaff = lines.split(" ")[0][1:]
                        if scaff_old != "Intial_value" and scaff_old != scaff:
                            error = "Y"
                        scaff_old = scaff
                    if "Sbjct" in lines:
                        if start_coor_switch == 0:
                            start_coor = int(lines.split(" ")[2])
                            start_coor_switch = 1
                        stop_coor =int(lines.split(" ")[-1][:-1])
                    if "Query" in lines and "=" not in lines:
                        if query_start_coor_switch == 0:
    #                        print(lines)
                            query_start_coor = int(lines.split(" ")[2])
                            query_start_coor_switch = 1
                        query_stop_coor =int(lines.split(" ")[-1][:-1])
        #                print (stop_coor)
                    
    #            print(query_name, start_coor, stop_coor, query_start_coor, query_stop_coor)
            if break_switch == 1:
                break
            
            if start_coor < stop_coor:
                complement = "0"
                length = (stop_coor-start_coor)/3
                start = start_coor
                stop = stop_coor
             
            elif start_coor > stop_coor:
                complement = "1"
                length = (-stop_coor+start_coor)/3
                start = stop_coor
                stop = start_coor
             
            else:
                error = "Y"
            print(length, query_length,query_length - 0.1*query_length )
            if length < query_length - 0.1*query_length:
                error = "Y"
    #        translated_sequence = (sequence_extractor(species_name.split("\n")[0], scaff, int(complement), start, stop ))
    #        print (translated_sequence[0])
            old_trans = ''
#             print(start, stop)
            while True and (start != 0 or stop != 0):
                if (query_name == "Exon_1" or query_name == query_name_list[-1]):
                    frame = 1
                    translated_sequence = (sequence_extractor(species_name.split(".")[1], scaff, int(complement), start, stop,frame ))
    #                    print(species_name.split("\n")[0], scaff, int(complement), start, stop,frame, query_name )
                    if translated_sequence == old_trans:
                        break
                    if query_name == "Exon_1" :
                        if translated_sequence[0] != "M":
                            print(translated_sequence)
#                             print(start, stop)
                            if complement == "0":
                                start = int(start) - 3
                            if complement == "1":
                                stop = int(stop) + 3
                            old_trans = translated_sequence
                        if "*" in translated_sequence:
                            error = "Y"
                            break
                        if translated_sequence[0] == "M":
    #                            print(translated_sequence)
    #                            print(start, stop)
                            break
    #                    elif query_name == "Exon1" and query_species in species_without_exon1:
    #                        print("Species_without_exon1")
    #                        if complement == "0":
    #                            start = int(start) - 3*(int(query_start_coor)-1)
    #                        if complement == "1":
    #                            stop = int(stop) + 3*(int(query_start_coor)-1)
    #                        break
                            
                    if query_name == query_name_list[-1]:
                        if translated_sequence[-1] != "*":
                            print(translated_sequence)
#                             print(start, stop)
                            if complement == "0":
                                stop = int(stop) + 3
                            if complement == "1":
                                start = int(start) - 3
                            old_trans = translated_sequence
                        if translated_sequence[-1] == "*":
    #                            print(translated_sequence)
    #                            print(start, stop)
                            break
                else:
                    break
            
            if (start != 0 or stop != 0):
                start_modifier = seq_modi[i][0]
                stop_modifier = seq_modi[i][1]
            else:
                start_modifier = 0
                stop_modifier = 0            
            seq_length = query_length
            if query_start_coor != "1" and query_name != "Exon_1":
                if complement == "0":
                    start = int(start) - 3*(int(query_start_coor)-1)
                if complement == "1":
                    stop = int(stop) + 3*(int(query_start_coor)-1)
    #        print (query_stop_coor, seq_length)
            if query_stop_coor != str(seq_length) and query_name != query_name_list[-1]:
                if complement == "0":
                    stop = int(stop) + 3*(int(seq_length)-int(query_stop_coor))
                if complement == "1":
                    start = int(start) - 3*(int(seq_length)-int(query_stop_coor))
    #        print(start, stop)
            if complement == "0":
                start = int(start) - int(start_modifier)
                stop = int(stop) +  int(stop_modifier)
                if old_end != 0 and old_end > stop:
                    
                    error = "Y"
                old_end = stop
                
            if complement == "1":
                start = int(start) - int(stop_modifier)
                stop = int(stop) +  int(start_modifier)
                if old_end != 0 and old_end < stop:
                    error = "Y"
                old_end = stop
    #        print(start, stop)
            
    #        print(translated_sequence)
    
            
            if start == 0 or stop == 0:
                error = "Y"
            output_format = str(species_name.split("\n")[0])+"," + str(scaff) +"," + str(start)+"," + str(stop)+"," + str(complement)+"," + str(error)+  ","+ str(query_name)+","+ str(query_start_coor)+","+str(query_stop_coor)+","+str(query_length)+ "\n"  
            Output_Sequence = Output_Sequence + output_format
        
        print(Output_Sequence)
        output_file = open("H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/1.Blast_output/"+species_name+f"/{species_name}_coordinates.csv",'w')
        output_file.write(Output_Sequence)
        output_file.close()
        #        break
    except:
        print(species_name[:-1], scaff, start_coor, stop_coor, complement, error)
        print("ERror")
##        break
# =============================================================================
# except:
#     
#     print(species_name[:-1], scaff, start_coor, stop_coor, complement, error)
# #    break
# =============================================================================
#print(Output_Sequence)
    

2.Galleria_mellonella
28.666666666666668 29 26.1
2.Galleria_mellonell NW_026442079.1 7100050 7099964 1 N
ERror


In [128]:
# rom sequence_extraction_called import *


# phylo_species = open("C:/Users/sauba/Desktop/Work_Stuff/MRJP/0.for_automation_temp_files/phylo_species.txt",'r')
# species_list = phylo_species.readlines()
species_list = ["2.Galleria_mellonella"]
#print(species_list)
query_name_list = ["Exon_1","Exon_2","Exon_3","Exon_4","Exon_5","Exon_6","Exon_7","Exon_8","Exon_9","Exon_10","Exon_11","Exon_12","Exon_13","Exon_14","Exon_15","Exon_16","Exon_17","Exon_18","Exon_19","Exon_20","Exon_21","Exon_22","Exon_23","Exon_24"]
#species_to_test = ["Amyelois_transitella","Bicyclus_anynana","Helicoverpa_armigera","Manduca_sexta","Zerene_cesonia",]
#query_length_list = [144,40,75,38,27,37,47]
seq_modi = [[0,2],[1,2],[1,1],[2,0],[0,0],[0,0],[0,1],[2,1],[2,2],[1,0],[0,0],[0,0],[0,2],[1,2],[1,2],[1,0],[0,1],[2,1],[2,1],[2,0],[0,0],[0,0],[0,0],[0,0]]
#query_length_list = [90,66,49,61,64,44,35]
#seq_modi = [[0,0,90],[0,0,66],[0,1,49],[2,2,61],[1,2,64],[1,0,44],[0,0,35],]
#try:
for species_name in species_list:
#     try:
    print(species_name)
    header = "Species," + "Scaffold," + "Start," + "Stop," + "Complement," + "Error," + "Gene,"+ "Query_start," + "Query_stop,"+ "Query_Length\n" 
    Output_Sequence = header
    scaff = "Intial_value"
    scaff_old = "Intial_value"
    old_end = 0
    for i in range(len(query_name_list)):
        query_name = query_name_list[i]
        Length_switch = "0"
#        query_length = query_length_list[i]
        tblast_out = open("H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/1.Blast_output/"+species_name+"/Galleria_mellonella_blast_out.txt",'r')
        lines_in_file = tblast_out.readlines()
        #print(lines_in_file)
        result_section_switch = 0
        start_coor_switch = 0
        query_start_coor_switch = 0
        stop_coor_switch = 0
        error = "N"
        break_switch = 0
#        scaff = ''
        start = 0
        stop = 0
        start_coor = 0
        stop_coor = 0

        for lines in lines_in_file:

            if query_name in lines:
                result_section_switch = 1
                query_species_split = lines.split(" ")[1].split("_")
                query_species = str(query_species_split[1]+"_"+query_species_split[2].rstrip())
#                     print(query_species)
#                     print(query_name)
#                break
            if result_section_switch == 1 and "Lambda" in lines:
                result_section_switch == 0
                break

#                if result_section_switch == 1 and query_species not in species_to_test:
#                    print(f"{species_name.rstrip()} has query {query_species}")
#                    break_switch = 1
#                    break

            if result_section_switch == 1:
                if "Length=" in lines and Length_switch == "0":
                    query_length = int(lines.split("=")[1][:-1])
                    Length_switch = 1
#                    print(query_length)
#                    break
                if ("Score" in lines or ">" in lines) and (start_coor_switch == 1):
    #                print (lines)
                    break
                if ">" in lines:

                    scaff = lines.split(" ")[0][1:]
                    if scaff_old != "Intial_value" and scaff_old != scaff:
                        error = "Y"
                    scaff_old = scaff
                if "Sbjct" in lines:
                    if start_coor_switch == 0:
                        start_coor = int(lines.split(" ")[2])
                        start_coor_switch = 1
                    stop_coor =int(lines.split(" ")[-1][:-1])
                if "Query" in lines and "=" not in lines:
                    if query_start_coor_switch == 0:
#                        print(lines)
                        query_start_coor = int(lines.split(" ")[2])
                        query_start_coor_switch = 1
                    query_stop_coor =int(lines.split(" ")[-1][:-1])
    #                print (stop_coor)

#            print(query_name, start_coor, stop_coor, query_start_coor, query_stop_coor)
        if break_switch == 1:
            break

        if start_coor < stop_coor:
            complement = "0"
            length = (stop_coor-start_coor)/3
            start = start_coor
            stop = stop_coor

        elif start_coor > stop_coor:
            complement = "1"
            length = (-stop_coor+start_coor)/3
            start = stop_coor
            stop = start_coor

        else:
            error = "Y"
        print(length, query_length,query_length - 0.2*query_length )
        if length < query_length - 0.2*query_length:
            error = "Y"
#        translated_sequence = (sequence_extractor(species_name.split("\n")[0], scaff, int(complement), start, stop ))
#        print (translated_sequence[0])
        old_trans = ''
#             print(start, stop)
        while True and (start != 0 or stop != 0):
            if (query_name == "Exon_1" or query_name == query_name_list[-1]):
                frame = 1
                translated_sequence = (sequence_extractor(species_name.split(".")[1], scaff, int(complement), start, stop,frame ))
#                    print(species_name.split("\n")[0], scaff, int(complement), start, stop,frame, query_name )
                if translated_sequence == old_trans:
                    break
                if query_name == "Exon_1" :
                    if translated_sequence[0] != "M":
                        print(translated_sequence)
#                             print(start, stop)
                        if complement == "0":
                            start = int(start) - 3
                        if complement == "1":
                            stop = int(stop) + 3
                        old_trans = translated_sequence
                    if "*" in translated_sequence:
                        error = "Y"
                        break
                    if translated_sequence[0] == "M":
#                            print(translated_sequence)
#                            print(start, stop)
                        break
#                    elif query_name == "Exon1" and query_species in species_without_exon1:
#                        print("Species_without_exon1")
#                        if complement == "0":
#                            start = int(start) - 3*(int(query_start_coor)-1)
#                        if complement == "1":
#                            stop = int(stop) + 3*(int(query_start_coor)-1)
#                        break

                if query_name == query_name_list[-1]:
                    if translated_sequence[-1] != "*":
                        print(translated_sequence)
#                             print(start, stop)
                        if complement == "0":
                            stop = int(stop) + 3
                        if complement == "1":
                            start = int(start) - 3
                        old_trans = translated_sequence
                    if translated_sequence[-1] == "*":
#                            print(translated_sequence)
#                            print(start, stop)
                        break
            else:
                break

        if (start != 0 or stop != 0):
            start_modifier = seq_modi[i][0]
            stop_modifier = seq_modi[i][1]
        else:
            start_modifier = 0
            stop_modifier = 0            
        seq_length = query_length
        if query_start_coor != "1" and query_name != "Exon_1":
            if complement == "0":
                start = int(start) - 3*(int(query_start_coor)-1)
            if complement == "1":
                stop = int(stop) + 3*(int(query_start_coor)-1)
#        print (query_stop_coor, seq_length)
        if query_stop_coor != str(seq_length) and query_name != query_name_list[-1]:
            if complement == "0":
                stop = int(stop) + 3*(int(seq_length)-int(query_stop_coor))
            if complement == "1":
                start = int(start) - 3*(int(seq_length)-int(query_stop_coor))
#        print(start, stop)
        if complement == "0":
            start = int(start) - int(start_modifier)
            stop = int(stop) +  int(stop_modifier)
            if old_end != 0 and old_end > stop:

                error = "Y"
            old_end = stop

        if complement == "1":
            start = int(start) - int(stop_modifier)
            stop = int(stop) +  int(start_modifier)
            if old_end != 0 and old_end < stop:
                error = "Y"
            old_end = stop
#        print(start, stop)

#        print(translated_sequence)


        if start == 0 or stop == 0:
            error = "Y"
        output_format = str(species_name.split("\n")[0])+"," + str(scaff) +"," + str(start)+"," + str(stop)+"," + str(complement)+"," + str(error)+  ","+ str(query_name)+","+ str(query_start_coor)+","+str(query_stop_coor)+","+str(query_length)+ "\n"  
        Output_Sequence = Output_Sequence + output_format

    print(Output_Sequence)
    output_file = open("H:/My Drive/Circadian Rhythm Genes Project/3.Genes from NCBI/1.Period/1.Blast_output/"+species_name+f"/{species_name}_coordinates.csv",'w')
    output_file.write(Output_Sequence)
    output_file.close()
        #        break
#     except:
#         print(species_name[:-1], scaff, start_coor, stop_coor, complement, error)
#         print("ERror")
##        break

2.Galleria_mellonella
28.666666666666668 29 23.2
28.666666666666668 27 21.6
28.666666666666668 51 40.8
20.666666666666668 31 24.8
18.666666666666668 26 20.8
43.666666666666664 44 35.2
23.666666666666668 25 20.0
66.66666666666667 67 53.6
72.66666666666667 75 60.0
36.666666666666664 37 29.6
39.666666666666664 40 32.0
49.666666666666664 50 40.0
38.666666666666664 39 31.2
12.666666666666666 29 23.2
47.666666666666664 50 40.0
61.666666666666664 63 50.4
46.666666666666664 49 39.2
32.666666666666664 37 29.6
24.666666666666668 38 30.4
53.666666666666664 61 48.8
37.666666666666664 67 53.6
40.666666666666664 45 36.0
33.666666666666664 34 27.2
62.666666666666664 83 66.4
PSLVNEQLGQLYVDLQLEGVASRLTLEEGITSSSSSGEETLTTTPKVRNYAFILNICNNYETF
PSLVNEQLGQLYVDLQLEGVASRLTLEEGITSSSSSGEETLTTTPKVRNYAFILNICNNYETFL
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
2.Galleria_mellonella,NW_026442079.1,7099962,7100050,1,N,Exon_1,1,29,29
2.Galleria_mellonella,NW_026442079.1,6,0,1,Y,